In [10]:
import pandas as pd
import numpy as np

from IPython.display import clear_output

from tqdm import tqdm
tqdm.pandas()

from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

In [11]:
data_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/back-to-basics-data'
folder = 'C:/Users/oryan/Documents/mergers-in-cosmos'
student_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/followup-data'
results_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/back-to-basics-results'
cosmos_folder = 'E:/cosmos-data'

In [12]:
df_prim = pd.read_csv(f'{data_folder}/cosmos-primary-data.csv', index_col = 0)[['ID', 'ALPHA_J2000', 'DELTA_J2000', 'ez_z_phot', 'lp_zBEST']]
df_sec = pd.read_csv(f'{data_folder}/cosmos-secondary-data-redone-2.csv', index_col = 0)[['Sec_ID', 'ALPHA_J2000', 'DELTA_J2000', 'ez_z_phot', 'lp_zBEST']]

In [18]:
def picking_z(ez_z, lp_z):
    if np.isnan(ez_z):
        if not np.isnan(lp_z):
            return lp_z
        else:
            return np.nan
    
    if np.isnan(lp_z):
        if not np.isnan(ez_z):
            return ez_z
        else:
            return np.nan
    
    if lp_z < ez_z:
        return lp_z
    elif ez_z <= lp_z:
        return ez_z
    else:
        return 'what'

In [19]:
df_prim_z = (
    df_prim
    .assign(redshift = df_prim.apply(lambda row: picking_z(row.ez_z_phot, row.lp_zBEST), axis = 1))
)
df_sec_z = (
    df_sec
    .assign(redshift = df_sec.apply(lambda row: picking_z(row.ez_z_phot, row.lp_zBEST), axis = 1))
)

In [24]:
df_merge = df_prim_z.merge(df_sec_z, left_index = True, right_index = True, how = 'left').dropna(subset=['redshift_x', 'redshift_y'])

In [25]:
df_merge

,ID,ALPHA_J2000_x,DELTA_J2000_x,ez_z_phot_x,lp_zBEST_x,redshift_x,Sec_ID,ALPHA_J2000_y,DELTA_J2000_y,ez_z_phot_y,lp_zBEST_y,redshift_y
4000705532984,857121,150.673667,2.226291,0.219858,0.2235,0.219858,846770.0,150.651543,2.226517,0.219908,0.8234,0.219908
4000705539529,1284864,149.686223,2.637412,0.705821,0.6984,0.698400,1298521.0,149.683847,2.650479,0.711060,0.6985,0.698500
4000707045178,265129,150.219590,1.668562,0.221175,0.2197,0.219700,271755.0,150.212648,1.667203,0.222554,0.2197,0.219700
4000707047768,246327,150.197217,1.631803,0.204897,0.8870,0.204897,260900.0,150.203052,1.663127,0.204859,1.0842,0.204859
4000707048117,238086,150.220247,1.636294,0.224496,0.2046,0.204600,252331.0,150.201283,1.655298,0.204635,NaN,0.204635
...,...,...,...,...,...,...,...,...,...,...,...,...
6000435703307,891936,150.540733,2.260768,0.193701,0.1595,0.159500,863206.0,150.500752,2.240888,0.159563,NaN,0.159563
6000443709301,1019667,150.097619,2.384458,0.076675,0.3141,0.076675,987251.0,150.027199,2.354704,0.076718,0.3694,0.076718
6000513404870,910711,150.519330,2.284680,0.028114,NaN,0.028114,826072.0,150.565350,2.207783,NaN,0.0281,0.028100
6000516624203,888830,150.443506,2.263871,0.038598,NaN,0.038598,821410.0,150.430508,2.203960,0.038621,NaN,0.038621


In [26]:
def calc_sep(ra1, dec1, ra2, dec2, z1, z2):
    d1 = cosmos.comoving_distance(z1).to(u.kpc)
    d2 = cosmos.comoving_distance(z2).to(u.kpc)
    
    c1 = SkyCoord(ra = ra1 * u.deg, dec = dec1 * u.deg, frame = 'fk5', distance = d1)
    c2 = SkyCoord(ra = ra2 * u.deg, dec = dec2 * u.deg, frame = 'fk5', distance = d1)
    
    sep_3d = c1.separation_3d(c2)
    return np.float64(sep_3d / u.kpc)

In [27]:
global cosmos
cosmos = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)

In [28]:
df_seps = (
    df_merge
    .assign(sep_3d = df_merge.apply(lambda row: calc_sep(row.ALPHA_J2000_x, row.DELTA_J2000_x, row.ALPHA_J2000_y, row.DELTA_J2000_y, row.redshift_x, row.redshift_y), axis = 1))
)

In [29]:
df_seps.sep_3d.max()

747.381880445241

In [30]:
df_seps.head()

,ID,ALPHA_J2000_x,DELTA_J2000_x,ez_z_phot_x,lp_zBEST_x,redshift_x,Sec_ID,ALPHA_J2000_y,DELTA_J2000_y,ez_z_phot_y,lp_zBEST_y,redshift_y,sep_3d
4000705532984,857121,150.673667,2.226291,0.219858,0.2235,0.219858,846770.0,150.651543,2.226517,0.219908,0.8234,0.219908,344.720901
4000705539529,1284864,149.686223,2.637412,0.705821,0.6984,0.698400,1298521.0,149.683847,2.650479,0.711060,0.6985,0.698500,579.706799
4000707045178,265129,150.219590,1.668562,0.221175,0.2197,0.219700,271755.0,150.212648,1.667203,0.222554,0.2197,0.219700,110.168984
4000707047768,246327,150.197217,1.631803,0.204897,0.8870,0.204897,260900.0,150.203052,1.663127,0.204859,1.0842,0.204859,464.721422
4000707048117,238086,150.220247,1.636294,0.224496,0.2046,0.204600,252331.0,150.201283,1.655298,0.204635,NaN,0.204635,390.962104


### Visual Inspection

In [31]:
def correct_sec(ra1, dec1, ra2, dec2):
        
    print(f'Primary Coords: {ra1}, {dec1}')
    print(f'Secondary Coords: {ra2}, {dec2}')
    
    correct = input(f'Is Secondary correct?')
    
    clear_output(wait = True)
    
    return correct

In [32]:
answers_dict = {}

In [37]:
df_dict = df_seps.to_dict(orient = 'index')
for i in list(df_dict.keys()):
    
    if i in list(answers_dict.keys()):
        continue
    
    ra1 = df_dict[i]['ALPHA_J2000_x']
    dec1 = df_dict[i]['DELTA_J2000_x']
    ra2 = df_dict[i]['ALPHA_J2000_y']
    dec2 = df_dict[i]['DELTA_J2000_y']
    answers_dict[i] = correct_sec(ra1,dec1,ra2,dec2)

Primary Coords: 150.33307778709744, 2.051833185632151
Secondary Coords: 150.33975073057763, 2.0499823001550026


KeyboardInterrupt: Interrupted by user

In [46]:
correct_sec_df = (
    df_quer
    .assign(sec_correct = df_quer.apply(lambda row: correct_sec(row.ALPHA_J2000_x, row.DELTA_J2000_x, row.ALPHA_J2000_y, row.DELTA_J2000_y), axis = 1))
)

Primary Coords: 150.28724578762234, 2.625417053462712
Secondary Coords: 150.2814192582384, 2.636284612824745


KeyboardInterrupt: Interrupted by user